In [27]:
import numpy as np
import random
from scipy.spatial import distance_matrix

In [18]:
n_vertices = int(input())
vertices = []
vert_ids = []

for _ in range(n_vertices):
    vertex_id, x_coord, y_coord = list(map(float, input().split()))
    vertex_id = int(vertex_id)
    
    vert_ids.append(vertex_id)
    vertices.append((x_coord, y_coord))
    
vertices = np.array(vertices)
vert_ids = np.array(vert_ids)

4
1 0.0 0.0
2 1.0 0.0
3 1.0 1.0
4 0.0 1.0


In [2]:
def get_dist(first_vert, second_vert):
    return np.linalg.norm(first_vert - second_vert)

def nearest_neighbour(vertices, start_ind):
    vert_count, _ = vertices.shape
    visited = np.zeros(vert_count, dtype=bool)
    ham_cycle = [start_ind]
    total_dist = 0

    for _ in range(visited.size-1):
        curr_vert = vertices[start_ind]
        visited[start_ind] = True

        min_dist = float('inf')
        nearest_vert_ind = -1

        for vert_ind in range(visited.size):
            vertex = vertices[vert_ind]
            dist = get_dist(vertex, curr_vert)

            if not visited[vert_ind] and dist < min_dist:
                min_dist = dist
                nearest_vert_ind = vert_ind

        total_dist += min_dist
        ham_cycle.append(nearest_vert_ind)
        start_ind = nearest_vert_ind

    ham_cycle.append(ham_cycle[0])
    return ham_cycle, total_dist

In [3]:
def nearest_insertion(vertices):
    vert_count, _ = vertices.shape
    visited = np.zeros(vert_count, dtype=bool)
    ham_cycle = []
    total_dist = 0
    
    # shortest edge
    shortest_length = float('inf')
    shortest_edge = None
    
    for i in range(vert_count):
        for j in range(i+1, vert_count):
            dist = get_dist(vertices[i], vertices[j])
            
            if dist < shortest_length:
                shortest_length = dist
                shortest_edge = [i, j]
                
    visited[shortest_edge] = True
    total_dist += shortest_length
    
    # smallest triangle with The edge
    triangle_vert = -1
    triangle_vert_length = float('inf')
    
    for i in range(vert_count):
        if not i in shortest_edge:
            curr_dist = get_dist(vertices[i], vertices[shortest_edge[0]]) + \
                        get_dist(vertices[i], vertices[shortest_edge[1]])
            
            if curr_dist < triangle_vert_length:
                triangle_vert_length = curr_dist
                triangle_vert = i
                                
    ham_cycle.extend([shortest_edge[0], triangle_vert, shortest_edge[1]])
    ham_cycle.append(shortest_edge[0])
    
    visited[triangle_vert] = True
    total_dist += triangle_vert_length
                
    for _ in range(vert_count-3):
        best_vert = -1
        shortest_diff = float('inf')
        insertion_ind = -1
        
        for new_cycle_vert in range(vert_count):
            if visited[new_cycle_vert]:
                continue
                
            for k in range(len(ham_cycle)-1):
                dist_diff = get_dist(vertices[ham_cycle[k]], vertices[new_cycle_vert]) + \
                            get_dist(vertices[ham_cycle[k+1]], vertices[new_cycle_vert]) - \
                            get_dist(vertices[ham_cycle[k]], vertices[ham_cycle[k+1]])
                
                if dist_diff < shortest_diff:
                    shortest_diff = dist_diff
                    best_vert = new_cycle_vert
                    insertion_ind = k
                            
        ham_cycle.insert(insertion_ind+1, best_vert)
        visited[best_vert] = True
        total_dist += shortest_diff
    
    return ham_cycle, total_dist

In [4]:
def local_search_2(vertices, ham_cycle, best_dist, max_time):
    start_time = time.time()
    vert_count, _ = vertices.shape
    
    while time.time() - start_time < max_time:
        is_changed = False
        
        for i in range(vert_count):
            for j in range(i+2, vert_count):
                e11 = vertices[ham_cycle[i - 1]]
                e12 = vertices[ham_cycle[i]]
                e21 = vertices[ham_cycle[j - 1]]
                e22 = vertices[ham_cycle[j]]
                
                old_dist = get_dist(e11, e12) + get_dist(e21, e22)
                new_dist = get_dist(e11, e21) + get_dist(e12, e22)

                if old_dist > new_dist:
                    is_changed = True
                    ham_cycle[i+1:j+1] = ham_cycle[i+1:j+1][::-1]
                    best_dist = best_dist + new_dist - old_dist
                    
        if not is_changed:
            return best_dist
        
    return best_dist

In [77]:
def read_data(filename):
    data = []
    
    with open(filename, "r") as fd:
        islands_cnt, unit_cost, k_islands, money_limit = list(map(int, fd.readline().split()))
        
        for line in fd.readlines():
            island_x, island_y, island_value = list(map(int, line.split()))
            data.append([island_x, island_y, island_value])
            
    return np.array(data), unit_cost, k_islands, money_limit

data, unit_cost, k_islands, money_limit = read_data("pirates_3.txt")

In [78]:
coordinates = data[:, :2]
values = data[:, -1]

pairwise_distances = distance_matrix(coordinates, coordinates)

def get_dist(i, j):
    return pairwise_distances[i, j]

In [79]:
def modified_nn(islands_coord, islands_values, unit_cost, 
                k_islands, money_limit, start_ind=0):
    vert_count, _ = islands_coord.shape
    visited = np.zeros(vert_count, dtype=bool)
    
    path = [start_ind]
    
    total_benefit = islands_values[start_ind]
    k_values = []

    for _ in range(visited.size-1):
        visited[start_ind] = True

        max_benefit = 0
        max_val = 0
        nearest_vert_ind = -1

        for vert_ind in range(visited.size):
            if visited[vert_ind]:
                continue

            val = islands_values[vert_ind]
            dist = get_dist(vert_ind, start_ind)
            
            travel_cost = dist * unit_cost
            benefit = val - travel_cost
            
            if benefit < 0 or val + sum(k_values) > money_limit:
                continue
            
            if benefit > max_benefit:
                max_benefit = benefit
                max_val = val
                nearest_vert_ind = vert_ind
                
        if nearest_vert_ind == -1:
            break
            
        k_values.append(max_val)
        
        if len(k_values) == k_islands:
            k_values.remove(k_values[0])

        total_benefit += max_benefit
        path.append(nearest_vert_ind)
        start_ind = nearest_vert_ind

    total_benefit -= unit_cost * get_dist(path[0], path[-1])
    path.append(path[0])
    
    return path, total_benefit

In [80]:
islands_cnt = data.shape[0]
max_benefit, best_path = 0, None
random_start_inds = [0] + random.sample(range(islands_cnt), 1)

for start_ind in random_start_inds:
    path, benefit = modified_nn(coordinates, values, unit_cost, 
                                k_islands, money_limit, start_ind)
    
    if benefit > max_benefit and path.count(0) > 0:
        best_path = path
        max_benefit = benefit
        
print(max_benefit)
print(*np.array(best_path)+1, sep=' ')

1058422.1955943946
1 470 226 32 254 304 428 182 661 544 195 196 97 952 913 750 147 588 125 11 472 657 364 545 426 12 557 530 338 957 286 534 6 481 910 973 647 914 856 757 851 595 679 308 881 947 161 828 818 413 353 93 76 945 10 821 207 577 250 869 239 285 449 515 152 35 639 658 201 720 673 707 94 686 185 243 294 494 119 271 314 766 573 101 608 732 642 546 926 850 367 471 781 803 832 570 225 618 665 272 644 754 409 715 37 59 259 714 667 756 628 581 430 563 8 221 452 929 13 703 587 805 394 206 164 414 432 596 175 7 610 918 669 335 14 814 906 134 834 190 124 760 619 920 795 567 696 521 553 482 654 967 554 606 681 464 79 569 771 377 591 616 925 122 889 812 274 738 875 5 150 549 572 643 804 210 739 693 291 203 516 205 407 336 439 347 935 231 172 298 466 724 575 697 219 40 897 943 704 912 535 307 469 249 78 158 63 16 592 552 214 838 880 344 50 345 648 393 438 232 123 611 180 9 457 113 460 789 787 790 370 620 310 238 748 220 558 467 496 91 17 263 680 403 505 458 126 135 257 82 670 191 723 22 

In [83]:
def modified_nn_2(islands_coord, islands_values, unit_cost, 
                k_islands, money_limit, start_ind=0):
    vert_count, _ = islands_coord.shape
    visited = np.zeros(vert_count, dtype=bool)
    
    path = [start_ind]
    
    total_benefit = islands_values[start_ind]
    k_values = []
    is_negative_benefit_ok = False

    while True:
        visited[start_ind] = True

        max_benefit = -unit_cost * get_dist(path[0], path[-1]) if is_negative_benefit_ok else 0
        max_val = 0
        nearest_vert_ind = -1

        for vert_ind in range(visited.size):
            if visited[vert_ind]:
                continue

            val = islands_values[vert_ind]
            dist = get_dist(start_ind, vert_ind)
            
            if is_negative_benefit_ok:
                dist += get_dist(vert_ind, path[0])
            
            travel_cost = dist * unit_cost
            benefit = val - travel_cost
            
            if benefit < 0 and is_negative_benefit_ok == False:
                continue
            
            if val + sum(k_values) > money_limit:
                continue
            
            if benefit > max_benefit:
                max_benefit = benefit
                max_val = val
                nearest_vert_ind = vert_ind
                
        if nearest_vert_ind == -1:
            if is_negative_benefit_ok:
                break
            else:
                is_negative_benefit_ok = True
                continue
        
        if is_negative_benefit_ok:
            max_benefit += unit_cost * get_dist(nearest_vert_ind, path[0])
            
        k_values.append(max_val)
        
        if len(k_values) == k_islands:
            k_values.remove(k_values[0])

        total_benefit += max_benefit
        path.append(nearest_vert_ind)
        start_ind = nearest_vert_ind             

    total_benefit -= unit_cost * get_dist(path[0], path[-1])
    path.append(path[0])
    
    return path, total_benefit

In [84]:
islands_cnt = data.shape[0]
max_benefit, best_path = 0, None
random_start_inds = [0] # + random.sample(range(islands_cnt), 10)

for start_ind in random_start_inds:
    path, benefit = modified_nn_2(coordinates, values, unit_cost, 
                                k_islands, money_limit, start_ind)
    
    if benefit > max_benefit and path.count(0) > 0:
        best_path = path
        max_benefit = benefit
        
print(max_benefit)
print(*np.array(best_path)+1, sep=' ')

1059629.3491651963
1 470 226 32 254 304 428 182 661 544 195 196 97 952 913 750 147 588 125 11 472 657 364 545 426 12 557 530 338 957 286 534 6 481 910 973 647 914 856 757 851 595 679 308 881 947 161 828 818 413 353 93 76 945 10 821 207 577 250 869 239 285 449 515 152 35 639 658 201 720 673 707 94 686 185 243 294 494 119 271 314 766 573 101 608 732 642 546 926 850 367 471 781 803 832 570 225 618 665 272 644 754 409 715 37 59 259 714 667 756 628 581 430 563 8 221 452 929 13 703 587 805 394 206 164 414 432 596 175 7 610 918 669 335 14 814 906 134 834 190 124 760 619 920 795 567 696 521 553 482 654 967 554 606 681 464 79 569 771 377 591 616 925 122 889 812 274 738 875 5 150 549 572 643 804 210 739 693 291 203 516 205 407 336 439 347 935 231 172 298 466 724 575 697 219 40 897 943 704 912 535 307 469 249 78 158 63 16 592 552 214 838 880 344 50 345 648 393 438 232 123 611 180 9 457 113 460 789 787 790 370 620 310 238 748 220 558 467 496 91 17 263 680 403 505 458 126 135 257 82 670 191 723 22 

In [85]:
# 2.88 + 0.53 + 0.22 + 0.93 = 4.56 -> Хор 7
# Отл 8: 5.3+

4.5600000000000005